<a href="https://colab.research.google.com/github/LukaLiegis/Stock-Price-Prediction/blob/main/TimeSeriesForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####Importing and Downloading Libraries

In [1]:
!pip install pandas_ta &> /dev/null
!pip install yfinance &> /dev/null
!pip install tensorflow_addons &> /dev/null

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf
import tensorflow as tf
import pandas_ta as ta

####Getting Data

In [3]:
data = yf.download("MSFT", period="max")
dataset = (data.drop(columns=['Adj Close']))

[*********************100%***********************]  1 of 1 completed


####Technical Indicators

Here I use mostly all of the indicators that Pandas_TA lets me.

In [4]:
def get_technical_indicators(dataset):
    # Create 7 and 21 days Moving Average
    dataset['ma7'] = dataset['Close'].rolling(window=7).mean()
    dataset['ma21'] = dataset['Close'].rolling(window=21).mean()
    dataset['ma100'] = dataset['Close'].rolling(window=100).mean()
    dataset['ma150'] = dataset['Close'].rolling(window=150).mean()
    dataset['ma200'] = dataset['Close'].rolling(window=200).mean()
    
    # Create Exponential moving average
    dataset["ema30"] = dataset.ta.ema(30)
    dataset['ema200'] = dataset.ta.ema(200)

    #Logarithmic Return
    dataset["Log_Data"] = (np.log(dataset["Close"]))

    #
    dataset["ebsw"] = dataset.ta.ebsw()
    
    #Other Pandas-TA Indicators
    dataset["RSI"] = dataset.ta.rsi()
    dataset["RSX"] = dataset.ta.rsx()
    dataset["uo"] = dataset.ta.uo()
    
    #Pandas-TA Statistics
    dataset["entropy"] = dataset.ta.entropy()
    dataset["kurtosis"] = dataset.ta.kurtosis()
    dataset["mad"] = dataset.ta.mad()
    dataset["median"] = dataset.ta.median()
    dataset["quantile"] = dataset.ta.quantile()
    dataset["skew"] = dataset.ta.skew()
    dataset["stdev"] = dataset.ta.stdev()
    dataset["variance"] = dataset.ta.variance()
    dataset["zscore"] = dataset.ta.zscore()

    #Pandas-TA Trend
    dataset["chop"] = dataset.ta.chop()
    dataset["decay"] = dataset.ta.decay()
    dataset["decreasing"] = dataset.ta.decreasing()
    dataset["dpo"] = dataset.ta.dpo()
    dataset["increasing"] = dataset.ta.increasing()
    dataset["qstick"] = dataset.ta.qstick()

    #Pandas-TA Volatility
    dataset["massi"] = dataset.ta.massi()

    #Pandas-TA Volume
    dataset["ad"] = dataset.ta.ad()
    
    return dataset

In [5]:
dataset_TI = get_technical_indicators(dataset)
dataset_TI = dataset_TI.dropna()

In [ ]:
#dataset_TI

####Feature Engineering

In [28]:
print('Total dataset has {} samples, and {} features.'.format(dataset_TI.shape[0], \
                                                              dataset_TI.shape[1]))


Total dataset has 9061 samples, and 34 features.


#####Scaling Data

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
names = dataset_TI.columns
d = scaler.fit_transform(dataset_TI)
scaled_df = pd.DataFrame(d, columns=names)
scaled_df

In [39]:
y = scaled_df.iloc[:, 3:4]
X = scaled_df.drop(scaled_df.iloc[:, 3:4],axis = 1)

In [42]:
#column_indices = {name: i for i, name in enumerate(scaled_df.columns)}

n = len(scaled_df)
#70% of the data is used for training.
X_train_df = X[0:int(n*0.7)]
y_train_df = y[0:int(n*0.7)]
#20% of the data is used for validation.
X_val_df = X[int(n*0.7):int(n*0.9)]
y_val_df = y[int(n*0.7):int(n*0.9)]
#10% of the data is being used for testing.
X_test_df = X[int(n*0.9):]
y_test_df = y[int(n*0.9):]

num_features = scaled_df.shape[1]

####Extracting high-level features with Stacked Autoencoders

####Model

In [ ]:
#creating stacked model
model=Sequential()
model.add(GRU(16,return_sequences=True,input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(PReLU())
model.add(Dropout(0.3))

model.add(GRU(4,return_sequences=False))
model.add(PReLU())
model.add(Dropout(0.3))

model.add(Dense(1))
model.add(PReLU())
model.compile(loss="huber", metrics=["mse", 'mape'],optimizer='adam')